In [15]:
import numpy as np
from scipy.io import loadmat
from google.colab import drive

drive.mount('/content/drive')
data = loadmat('/content/drive/MyDrive/iris_shuffled.mat')
X = data['iris_data'].astype(float)  # 문자열로 되어 있을 수 있어서 float으로 바꿈
y = data['iris_class'].flatten()     # (150, 1) 모양일 수 있어서 1차원으로 펼침

# 데이터를 섞어서 70%는 학습용, 30%는 테스트용으로 사용
def train_test_split(X, y, test_ratio=0.3):
    n_samples = X.shape[0]
    indices = np.arange(n_samples)
    np.random.seed(42)  # 결과 재현 가능하도록 고정
    np.random.shuffle(indices)  # 인덱스를 무작위로 섞음
    split = int(n_samples * (1 - test_ratio))  # 학습 데이터 개수
    train_idx, test_idx = indices[:split], indices[split:]
    return X[train_idx], X[test_idx], y[train_idx], y[test_idx]

X_train, X_test, y_train, y_test = train_test_split(X, y)

# 두 벡터 간 거리를 계산
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2)**2))

# x_test 데이터와 학습데이터 간의 거리를 모두 계산한 뒤,
# 가까운 K개의 데이터를 고르고, 그 중 가장 많이 등장한 클래스를 예측값으로
def knn_predict(X_train, y_train, x_test, k):
    distances = [euclidean_distance(x_test, x) for x in X_train]  # 모든 학습 샘플과 거리 계산
    k_indices = np.argsort(distances)[:k]  # 가장 가까운 k개 찾기
    k_labels = [y_train[i] for i in k_indices]  # 라벨 모으기
    values, counts = np.unique(k_labels, return_counts=True)  # 라벨별로 몇 개씩 있는지 세기
    return values[np.argmax(counts)]  # 가장 많이 등장한 라벨을 결과로 반환

# 학습 데이터에 대해 잘 분류했는지를 계산
k_values = [5, 10, 15, 20, 25, 30]
results = []

for k in k_values:
    y_pred = []  # 예측 결과 저장할 리스트
    for x in X_train:
        pred = knn_predict(X_train, y_train, x, k)  # 학습 데이터로 학습 데이터를 예측
        y_pred.append(pred)
    y_pred = np.array(y_pred)
    error_rate = np.mean(y_pred != y_train)  # 오분류율 = 예측과 정답이 다른 비율
    results.append((k, error_rate))  # 결과 저장

print("K 값에 따른 학습 오분류율")
for k, error in results:
    print(f"K = {k:2} → 오분류율 = {error:.4f}")

Mounted at /content/drive


<ipython-input-15-916231a73e6b>:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  X = data['iris_data'].astype(float)  # 문자열로 되어 있을 수 있어서 float으로 바꿈


K 값에 따른 학습 오분류율
K =  5 → 오분류율 = 0.0190
K = 10 → 오분류율 = 0.0095
K = 15 → 오분류율 = 0.0095
K = 20 → 오분류율 = 0.0190
K = 25 → 오분류율 = 0.0381
K = 30 → 오분류율 = 0.0667


In [16]:
import numpy as np
from scipy.io import loadmat
from google.colab import drive

drive.mount('/content/drive')
data = loadmat('/content/drive/MyDrive/iris_shuffled.mat')
X = data['iris_data'].astype(float)  # 문자열로 되어 있을 수 있어서 float으로 바꿈
y = data['iris_class'].flatten()     # (150, 1) 모양일 수 있어서 1차원으로 펼침

# 인덱스를 무작위로 섞은 뒤 앞에서 120개는 학습, 나머지 30개는 테스트로 설정
def train_test_split_fixed(X, y, train_size=120):
    indices = np.arange(X.shape[0])  # 전체 데이터 인덱스 (0~149)
    np.random.seed(42)               # 실험 재현을 위해 seed 고정
    np.random.shuffle(indices)       # 인덱스 무작위 섞기

    train_idx = indices[:train_size]  # 앞에서 120개는 학습용
    test_idx = indices[train_size:]  # 나머지 30개는 테스트용

    # 나눈 인덱스를 사용해 실제 데이터를 선택해 반환
    return X[train_idx], X[test_idx], y[train_idx], y[test_idx]

# 실제 학습/테스트 데이터를 나눔
X_train, X_test, y_train, y_test = train_test_split_fixed(X, y)

# 두 벡터간의 유클리디안 거리를 계산
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2)**2))

# x_test: 분류하고 싶은 샘플
# X_train, y_train: 학습 데이터
# k: 주변 몇 개를 볼지?
def knn_predict(X_train, y_train, x_test, k):
    # 학습 데이터 각각과의 거리 계산
    distances = [euclidean_distance(x_test, x) for x in X_train]

    # 거리가 가까운 순으로 정렬한 인덱스 중 앞의 k개를 선택
    k_indices = np.argsort(distances)[:k]

    # 선택된 k개의 이웃 데이터들의 클래스 라벨 모으기
    k_labels = [y_train[i] for i in k_indices]

    # 가장 많이 등장한 라벨을 최종 예측값으로 선택
    values, counts = np.unique(k_labels, return_counts=True)
    return values[np.argmax(counts)]

# K값들에 대해 반복하며 결과 비교
k_values = [5, 10, 15, 20, 25, 30]
results = []

# 각각의 K에 대해 오분류율을 계산해 저장
for k in k_values:
    # 학습 데이터에 대해 자기 자신을 예측
    y_train_pred = [knn_predict(X_train, y_train, x, k) for x in X_train]
    train_error = np.mean(y_train_pred != y_train)  # 정답과 비교해서 틀린 비율

    # 테스트 데이터에 대해 예측
    y_test_pred = [knn_predict(X_train, y_train, x, k) for x in X_test]
    test_error = np.mean(y_test_pred != y_test)  # 테스트 오분류율

    # K값과 오분류율을 튜플로 저장
    results.append((k, train_error, test_error))

print("K 값에 따른 오분류율 (학습 / 테스트)")
print(" K | 학습 오분류율 | 테스트 오분류율")
print("---|---------------|----------------")
for k, e_train, e_test in results:
    print(f"{k:2d} |     {e_train:.4f}    |      {e_test:.4f}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-16-97703de5f747>:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  X = data['iris_data'].astype(float)  # 문자열로 되어 있을 수 있어서 float으로 바꿈


K 값에 따른 오분류율 (학습 / 테스트)
 K | 학습 오분류율 | 테스트 오분류율
---|---------------|----------------
 5 |     0.0167    |      0.0667
10 |     0.0083    |      0.0667
15 |     0.0083    |      0.0667
20 |     0.0167    |      0.0667
25 |     0.0250    |      0.0667
30 |     0.0417    |      0.0667
